<a href="https://colab.research.google.com/github/asadovkamran/jiguli_classification/blob/main/jiguli_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
DATA_DIR = r"sample_data/dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
VAL_SPLIT = 0.2
SEED = 42

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

print("Classes:", train_ds.class_names)

Found 210 files belonging to 2 classes.
Using 168 files for training.
Found 210 files belonging to 2 classes.
Using 42 files for validation.
Classes: ['jiguli', 'not-jiguli']


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
])

In [8]:
def prepare(ds, augment=False):
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE
        )

    ds = ds.map(
        lambda x, y: (preprocess_input(x), y),
        num_parallel_calls=AUTOTUNE
    )

    return ds.prefetch(AUTOTUNE)

In [9]:
def predict_image(model, img_path):
    img = cv2.imread(img_path)
    if img is None:
        print("Could not read image:", img_path)
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_resized = cv2.resize(img_rgb, (224, 224))

    img_array = np.array(img_resized, dtype=np.float32)

    img_pre = preprocess_input(img_array)

    img_pre = np.expand_dims(img_pre, axis=0)

    prob = model.predict(img_pre)[0][0]

    label = "jiguli" if prob < 0.5 else "not-jiguli"

    print(f"Prediction: {label} (prob={prob:.4f})")
    return label, prob

In [10]:
train_ds_prep = prepare(train_ds, augment=True)
val_ds_prep   = prepare(val_ds, augment=False)

In [11]:
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
base_model.trainable = True

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 2,225,153 (8.49 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [13]:
history = model.fit(
    train_ds_prep,
    validation_data=val_ds_prep,
    epochs=12
)

Epoch 1/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.5324 - loss: 0.7516 - val_accuracy: 0.7381 - val_loss: 0.6392
Epoch 2/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.6896 - loss: 0.5988 - val_accuracy: 0.7143 - val_loss: 0.6135
Epoch 3/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.7222 - loss: 0.5431 - val_accuracy: 0.7143 - val_loss: 0.5871
Epoch 4/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8017 - loss: 0.4757 - val_accuracy: 0.7381 - val_loss: 0.5618
Epoch 5/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.8417 - loss: 0.4168 - val_accuracy: 0.7857 - val_loss: 0.5407
Epoch 6/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.9044 - loss: 0.3752 - val_accuracy: 0.8333 - val_loss: 0.5219
Epoch 7/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.8372 - loss: 0.3760 - val_accuracy: 0.8095 - val_loss: 0.5053
Epoch 8/12
11/11 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.9433 - loss: 0.2933 - val_accuracy: 0.8571 - val_loss:

In [14]:
val_loss, val_acc = model.evaluate(val_ds_prep)
print("Validation loss:", val_loss)
print("Validation accuracy:", val_acc)

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 437ms/step - accuracy: 0.8858 - loss: 0.4224
Validation loss: 0.42832523584365845
Validation accuracy: 0.8809523582458496


In [15]:
y_true = []
for images, labels in val_ds_prep:
    y_true.append(labels.numpy())
y_true = np.concatenate(y_true)

y_pred_prob = model.predict(val_ds_prep).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

print("\nClassification report:\n")
print(classification_report(y_true, y_pred, target_names=train_ds.class_names))

3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step
Confusion matrix:
 [[ 3  8]
 [ 7 24]]

Classification report:

              precision    recall  f1-score   support

      jiguli       0.30      0.27      0.29        11
  not-jiguli       0.75      0.77      0.76        31

    accuracy                           0.64        42
   macro avg       0.53      0.52      0.52        42
weighted avg       0.63      0.64      0.64        42

